# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-2.36,72,21,9.44,GS,1729713677
1,1,udachny,66.4167,112.4000,-15.53,98,99,1.21,RU,1729713677
2,2,karachi,24.9056,67.0822,25.90,61,0,2.06,PK,1729713499
3,3,oltinko'l,40.8011,72.1634,12.86,79,95,1.17,UZ,1729713677
4,4,puerto deseado,-47.7503,-65.8938,11.26,60,11,5.75,AR,1729713677


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 3.98)]
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 26.94) & (city_data_df["Max Temp"] > 21.12)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,karachi,24.9056,67.0822,25.90,61,0,2.06,PK,1729713499
26,26,bethel,41.3712,-73.4140,24.77,59,0,1.34,US,1729713680
82,82,tamanrasset,22.7850,5.5228,25.95,19,0,3.09,DZ,1729713685
120,120,newman,37.3138,-121.0208,25.02,35,0,1.79,US,1729713688
160,160,sanya,18.2431,109.5050,24.09,73,0,1.00,CN,1729713693
169,169,san sebastian de la gomera,28.0916,-17.1133,22.20,78,0,2.06,ES,1729713694
260,260,tshabong,-26.0500,22.4500,22.00,12,0,3.75,BW,1729713702
394,394,martinsville,36.6915,-79.8725,25.90,30,0,2.57,US,1729713717
440,440,marovoay,-16.1000,46.6333,26.22,46,0,3.12,MG,1729713722
486,486,khuzdar,27.7384,66.6434,21.43,27,0,0.49,PK,1729713649


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

data = {
    "City": city,
    "Country": country,
    "Lat": lat,
    "Lng": lng,
    "Humidity": humidity
}

hotel_df = pd.DataFrame(data)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,karachi,PK,24.9056,67.0822,61,
26,bethel,US,41.3712,-73.4140,59,
82,tamanrasset,DZ,22.7850,5.5228,19,
120,newman,US,37.3138,-121.0208,35,
160,sanya,CN,18.2431,109.5050,73,
169,san sebastian de la gomera,ES,28.0916,-17.1133,78,
260,tshabong,BW,-26.0500,22.4500,12,
394,martinsville,US,36.6915,-79.8725,30,
440,marovoay,MG,-16.1000,46.6333,46,
486,khuzdar,PK,27.7384,66.6434,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
karachi - nearest hotel: Quetta Agha Hotel
bethel - nearest hotel: Hampton Inn Danbury
tamanrasset - nearest hotel: فندق الأمان
newman - nearest hotel: No hotel found
sanya - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
tshabong - nearest hotel: No hotel found
martinsville - nearest hotel: Baymont Martinsville
marovoay - nearest hotel: No hotel found
khuzdar - nearest hotel: Al-Mustafa hotel
monterey - nearest hotel: No hotel found
vicuna - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
2,karachi,PK,24.9056,67.0822,61,Quetta Agha Hotel
26,bethel,US,41.3712,-73.4140,59,Hampton Inn Danbury
82,tamanrasset,DZ,22.7850,5.5228,19,فندق الأمان
120,newman,US,37.3138,-121.0208,35,No hotel found
160,sanya,CN,18.2431,109.5050,73,No hotel found
169,san sebastian de la gomera,ES,28.0916,-17.1133,78,Hotel Villa Gomera
260,tshabong,BW,-26.0500,22.4500,12,No hotel found
394,martinsville,US,36.6915,-79.8725,30,Baymont Martinsville
440,marovoay,MG,-16.1000,46.6333,46,No hotel found
486,khuzdar,PK,27.7384,66.6434,27,Al-Mustafa hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    frame_height = 600,
    frame_width = 800,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)